<a href="https://colab.research.google.com/github/MichaelKru92/Projekt-ML-Modelierung/blob/main/Erstellen_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/MichaelKru92/Projekt-ML-Modelierung.git

Cloning into 'Projekt-ML-Modelierung'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 64 (delta 11), reused 2 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 3.70 MiB | 7.60 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [6]:
import pandas as pd
import sqlite3
from sqlite3 import Error
import csv
#-----------------------------------file path-------------------------------------------#
material_path = "/content/Projekt-ML-Modelierung/Datenbank"
# source database path
DB_FILE_PATH = f"{material_path}/source_metabolic_syndrome_disease.db"
# folder contains csv files
csv_path = f"{material_path}/metabolic_syndrome_disease/"
print(DB_FILE_PATH, csv_path)
#-----------------------------------SQL tables------------------------------------------#
# dictionary key: name of table, values: sql query to create table
sql_table = {}
# sql query to create disease table
sql_table["disease"] = """CREATE TABLE IF NOT EXISTS disease (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)

                       );"""
# sql query to create organizations table
sql_table["organizations"] = """CREATE TABLE IF NOT EXISTS organizations (
                            Id STRING PRIMARY KEY,
                            NAME STRING,
                            ADDRESS STRING,
                            CITY STRING,
                            STATE STRING,
                            ZIP STRING,
                            LAT NUMERIC,
                            LON NUMERIC,
                            PHONE STRING,
                            REVENUE NUMERIC,
                            UTILIZATION INTEGER
                    );"""
# sql query to create providers table
sql_table["providers"] = """CREATE TABLE IF NOT EXISTS providers (
                            Id STRING PRIMARY KEY,
                            ORGANIZATION STRING,
                            NAME STRING,
                            GENDER STRING,
                            SPECIALITY STRING,
                            ADDRESS STRING,
                            CITY STRING,
                            STATE STRING,
                            ZIP STRING,
                            LAT NUMERIC,
                            LON NUMERIC,
                            UTILIZATION INTEGER,
                            FOREIGN KEY (ORGANIZATION)
                                REFERENCES organizations (Id)
                            );"""
# sql query to create payers table
sql_table["payers"] = """CREATE TABLE IF NOT EXISTS payers (
                            Id STRING PRIMARY KEY,
                            NAME STRING,
                            ADDRESS STRING,
                            CITY STRING,
                            STATE_HEADQUATERED STRING,
                            ZIP STRING,
                            PHONE STRING,
                            AMOUNT_COVERED NUMERIC,
                            AMOUNT_UNCOVERED NUMERIC,
                            REVENUE INTEGER,
                            COVERED_ENCOUNTERS INTEGER,
                            UNCOVERED_ENCOUNTERS INTEGER,
                            COVERED_MEDICATIONS INTEGER,
                            UNCOVERED_MEDICATIONS INTEGER,
                            COVERED_PROCEDURES INTEGER,
                            UNCOVERED_PROCEDURES INTEGER,
                            COVERED_IMMUNIZATIONS INTEGER,
                            UNCOVERED_IMMUNIZATIONS INTEGER,
                            UNIQUE_CUSTOMERS INTEGER,
                            QOLS_AVG NUMERIC,
                            MEMBER_MONTHS INTEGER
                            );"""
# sql query to create payer_transitions table
sql_table["payer_transitions"] = """CREATE TABLE IF NOT EXISTS payer_transitions (
                            PATIENT STRING,
                            START_YEAR INTEGER,
                            END_YEAR INTEGER,
                            PAYER STRING,
                            OWNERSHIP STRING,
                            FOREIGN KEY (PATIENT)
                                REFERENCES patients (Id)
                            FOREIGN KEY (PAYER)
                                REFERENCES payers (Id)
)"""
#  sql query to create patients table
sql_table["patients"] = """CREATE TABLE IF NOT EXISTS patients (
                           Id STRING PRIMARY KEY,
                           BIRTHDATE DATE,
                           DEATHDATE DATE,
                           SSN STRING,
                           DRIVERS STRING,
                           PASSPORT STRING,
                           PREFIX STRING,
                           FIRST STRING,
                           LAST STRING,
                           SUFFIX STRING,
                           MAIDEN STRING,
                           MARITAL STRING,
                           RACE STRING,
                           ETHNICITY STRING,
                           GENDER STRING,
                           BIRTHPLACE STRING,
                           ADDRESS STRING,
                           CITY STRING,
                           STATE STRING,
                           COUNTRY STRING,
                           ZIP STRING,
                           LAT INTEGER,
                           LON INTEGER,
                           HEALTHCARE_EXPENSES INTEGER,
                           HEALTHCARE_COVERAGE INTEGER
                       );"""
# sql query to create encounters table
sql_table["encounters"] = """CREATE TABLE IF NOT EXISTS encounters (
                            Id STRING PRIMARY KEY,
                            START DATE,
                            STOP DATE,
                            PATIENT STRING,
                            ORGANIZATIONS STRING,
                            PROVIDER STRING,
                            PAYER STRING,
                            ENCOUNTERCLASS STRING,
                            CODE STRING,
                            DESCRIPTION STRING,
                            BASE_ENCOUNTER_COST INTEGER,
                            TOTAL_CLAIM_COST INTEGER,
                            PAYER_COVERAGE INTEGER,
                            REASONCODE STRING,
                            REASONDESCRIPTION STRING,
                            FOREIGN KEY (PATIENT)
                                REFERENCES patients (Id)
                            FOREIGN KEY (ORGANIZATIONS)
                                REFERENCES organizations (Id)
                            FOREIGN KEY (PROVIDER)
                                REFERENCES providers (Id)
                            FOREIGN KEY (PAYER)
                                REFERENCES payers (Id)
                       );"""
# sql query to create careplans table
sql_table["careplans"] = """CREATE TABLE IF NOT EXISTS careplans (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                       );"""
# sql query to create conditions table
sql_table["conditions"] = """CREATE TABLE IF NOT EXISTS conditions (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                       );"""
# sql query to create medications table
sql_table["medications"] = """CREATE TABLE IF NOT EXISTS medications (
                            START DATE,
                            STOP DATE,
                            PATIENT STRING,
                            PAYER STRING,
                            ENCOUNTER STRING,
                            CODE STRING,
                            DESCRIPTION STRING,
                            BASE_COST INTEGER,
                            PAYER_COVERAGE INTEGER,
                            DISPENSES INTEGER,
                            TOTALCOST INTEGER,
                            REASONCODE STRING,
                            REASONDESCRIPTION STRING,
                            FOREIGN KEY (PATIENT)
                                REFERENCES patients (Id)
                            FOREIGN KEY (ENCOUNTER)
                                REFERENCES encounters (Id)
                            FOREIGN KEY (PAYER)
                                REFERENCES payers (Id)
                       );"""
# sql query to create procedures table
sql_table["procedures"]= """CREATE TABLE IF NOT EXISTS procedures (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                       );"""
# sql query to create observations table
sql_table["observations"]= """CREATE TABLE IF NOT EXISTS observations (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           VALUE STRING,
                           UNITS STRING,
                           TYPE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                       );"""
# sql query to create devices table
sql_table["devices"] = """CREATE TABLE IF NOT EXISTS devices (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           UDI STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                       );"""
# sql query to create Imaging_studies table
sql_table["imaging_studies"] = """CREATE TABLE IF NOT EXISTS imaging_studies (
                           Id STRING PRIMARY KEY,
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           BODYSITE_CODE STRING,
                           BODYSITE_DESCRIPTION STRING,
                           MODALITY_CODE STRING,
                           MODALITY_DESCRIPTION STRING,
                           SOP_CODE STRING,
                           SOP_DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                       );"""
# sql query to create Immunizations table
sql_table["immunizations"]= """CREATE TABLE IF NOT EXISTS immunizations(
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                       );"""

#--------------------------functions to import csv in SQL DB-----------------------#
def connect_to_db(db_file):
    """
    Connect to an SQlite database, if db file does not exist it will be created
    :param db_file: absolute or relative path of db file
    :return: sqlite3 connection
    """
    sqlite3_conn = None

    try:
        sqlite3_conn = sqlite3.connect(db_file)
        return sqlite3_conn

    except Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()
#-------------------------------------------------------------------------------------------#
def open_csv_file(csv_file_path):
    """
    Open and read data from a csv file without headers (skipping the first row)
    :param csv_file_path: path of the csv file to process
    :return: a list with the csv content
    """
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)

        data = list()
        for row in reader:
            data.append(row)

        return data
#-------------------------------------------------------------------------------------------#
def get_column_names_from_db_table(sql_cursor, table_name):
    """
    Scrape the column names from a database table to a list and convert to a comma separated string, count the number
    of columns in a database table
    :param sql_cursor: sqlite cursor
    :param table_name: table name to get the column names from
    :return: a comma separated string with column names, an integer with number of columns
    """

    table_column_names = 'PRAGMA table_info(' + table_name + ');'
    sql_cursor.execute(table_column_names)
    table_column_names = sql_cursor.fetchall()

    column_count = len(table_column_names)

    column_names = list()

    for name in table_column_names:
        column_names.append(name[1])

    return ', '.join(column_names), column_count
#-------------------------------------------------------------------------------------------#
def insert_values_to_table(cursor, table_name, csv_file_path):

    """
    Open a csv file, store its content in a list excluding header and insert the data from the list to db table
    :param table_name: table name in the database to insert the data into
    :param csv_file_path: path of the csv file to process
    :return: None
    """

    # Read CSV file content
    values_to_insert = open_csv_file(csv_file_path)

    # Insert to table
    if len(values_to_insert) > 0:
        column_names, column_numbers = get_column_names_from_db_table(cursor, table_name)

        values_str = '?,' * column_numbers
        # print("*****", values_str, column_names, column_numbers)
        values_str = values_str[:-1]

        sql_query = 'INSERT OR REPLACE INTO ' + table_name + '(' + column_names + ') VALUES (' + values_str + ')'


        cursor.executemany(sql_query, values_to_insert)
        # conn.commit()

        print('SQL insert process finished')
    else:
        print('Nothing to insert')

        # conn.close()

    # else:
        # print('Connection to database failed')

#--------------------------------------csv import----------------------------------------------#
# list of tables
sql_table.keys()

# import csv into db
conn = connect_to_db(DB_FILE_PATH)
if conn is not None:
        cursor = conn.cursor()
        for name in sql_table.keys():
          csv_file = csv_path + name + ".csv"
          print(name, csv_file)

          cursor.execute(sql_table[name])

          insert_values_to_table(cursor, name, csv_file)
        # conn.commit()
else:
        print('Connection to database failed')


## sql query to create Index on Column CODE in table immunizations
cursor.execute(f"CREATE INDEX index_immunization ON immunizations (CODE);")

## check data in a table
cursor.execute("SELECT * FROM payers LIMIT 5;")
rows = cursor.fetchall()
for row in rows:
    print(row)


conn.commit()
conn.close()

/content/Projekt-ML-Modelierung/Datenbank/source_metabolic_syndrome_disease.db /content/Projekt-ML-Modelierung/Datenbank/metabolic_syndrome_disease/
disease /content/Projekt-ML-Modelierung/Datenbank/metabolic_syndrome_disease/disease.csv
Nothing to insert
organizations /content/Projekt-ML-Modelierung/Datenbank/metabolic_syndrome_disease/organizations.csv
SQL insert process finished
providers /content/Projekt-ML-Modelierung/Datenbank/metabolic_syndrome_disease/providers.csv
SQL insert process finished
payers /content/Projekt-ML-Modelierung/Datenbank/metabolic_syndrome_disease/payers.csv
SQL insert process finished
payer_transitions /content/Projekt-ML-Modelierung/Datenbank/metabolic_syndrome_disease/payer_transitions.csv
SQL insert process finished
patients /content/Projekt-ML-Modelierung/Datenbank/metabolic_syndrome_disease/patients.csv
SQL insert process finished
encounters /content/Projekt-ML-Modelierung/Datenbank/metabolic_syndrome_disease/encounters.csv
SQL insert process finished
